In [ ]:
#Import the data from understat.com using the code below 

import asyncio
import json

import aiohttp

import pandas as pd
import os
from understat import Understat


async def main():
    leagues = ["EPL","La liga","Bundesliga", "Serie A", "Ligue 1"]
    years = [2021]
    
    try:
        for f in leagues:
            for z in years:
                os.mkdir("Understat/"+f+"/"+str(z)+"/")
    except FileExistsError:
        pass
    
    async with aiohttp.ClientSession() as session:
        data = Understat(session)
        for i in range(len(leagues)):
            for j in range(len(years)):
                results = await data.get_league_results(leagues[i], years[j])
                new_results = "["
                for q in range(len(results)):
                    tst = str(results[q]).replace(" 'h': {","")
                    tst1 = tst.replace("}, 'a': {",",")
                    tst2 = tst1.replace("}, 'goals': {",",")
                    tst3 = tst2.replace("}, 'xG': {",",")
                    tst4 = tst3.replace("}, 'datetime'",", 'datetime'")
                    tst5 = tst4.replace(", 'forecast': {",",")
                    tst6 = tst5.replace("}}","},")
                    
                    ##replacing id duplicates 
                    tst7 = tst6.replace("'id'","'MatchID'",1).replace("'id'","'HomeID'",1).replace("'id'","'AwayID'",1)

                    ##replacing title
                    tst8 = tst7.replace("'title'","'HomeTeam'",1).replace("'title'","'AwayTeam'",1)

                    ##replacing short_title
                    tst9 = tst8.replace("'short_title'","'Home_short'",1).replace("'short_title'","'Away_short'",1)

                    ##replacing h
                    tst10 = tst9.replace("'h'","'home_goals'",1).replace("'h'","'home_xG'",1)

                    ##replacing a
                    tst11 = tst10.replace("'a'","'away_goals'",1).replace("'a'","'away_xG'",1)
                    
                    new_results += tst11
                new_results += "]"
                file = "Understat/"+leagues[i]+"/"+str(years[j])+"/"+str(years[j])
                with open(file+".json",'w') as outfile:
                    json.dump(eval(new_results),outfile)
                df = pd.read_json(file+".json")
                df.to_csv(file+".csv", index = None)
            
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

#The data from 538 was downloaded from their github link - https://github.com/fivethirtyeight/data/tree/master/soccer-spi




In [22]:
#Organize data scrapped from understat and SPI

import pandas as pd
import pandasql as ps
import csv

loc1 = "538/spi_matches.csv"
######################################
# ligue 1 - 1843
# BPL - 2411
# La Liga - 1869
# Serie A - 1854
# Bundesliga - 1845
######################################

loc2 = "Understat/"


understat_data = []

years = [2016, 2017, 2018, 2019, 2020,2021]

league = ["Bundesliga","EPL","La liga","Ligue 1","Serie A"]


In [23]:
#Understat data does not have results so add results for home, away and draw

import pandas as pd

for q in range(len(league)):
    for k in range(len(years)):
        result = []
        df = pd.read_csv(loc2 +league[q]+"/"+str(years[k])+"/"+ str(years[k])+".csv")
        with open(loc2 +league[q]+"/"+str(years[k])+"/"+ str(years[k])+".csv", newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for column in reader:
                HG = column['home_goals']
                AG = column['away_goals']
                if(HG > AG):
                    result.append('H')
                elif(HG < AG):
                    result.append('A')
                elif(HG == AG):
                    result.append('D')
        try:
            year_s = [str(years[k])]*len(result)
            df.insert(1,"Season",year_s)
            df.insert(17,"Results",result)
            df.to_csv(loc2 +league[q]+"/"+str(years[k])+"/"+ str(years[k])+".csv")
        except ValueError:
            df.drop('Season', inplace=True, axis=1)
            df.drop('Results', inplace=True, axis=1)
            year_s = [str(years[k])]*len(result)
            df.insert(1,"Season",year_s)
            df.insert(17,"Results",result)
            df.to_csv(loc2 +league[q]+"/"+str(years[k])+"/"+ str(years[k])+".csv")

#save total processed understat data into a single file 
for i in range(len(league)):
    for j in range(len(years)):
        understat_data.append(pd.read_csv(loc2 +league[i]+"/"+str(years[j])+"/"+ str(years[j])+".csv"))


understat_data = pd.concat(understat_data, ignore_index = True)

print(understat_data.shape)

understat_data.to_csv("Understat/understat_processed.csv")

(9662, 38)


In [25]:
#Use league ids to only collect top 5 league data from 538
league_id = [1843, 2411, 1869, 1854, 1845]

df = pd.read_csv(loc1)

#Sql queury to only get data from top 5 leagues
q1 = """SELECT * FROM df
        WHERE league_id IN (1843, 2411, 1869, 1854, 1845)"""

df1 = ps.sqldf(q1)

#Some games were not played due to covid remove them from dataset
q2 = """SELECT * FROM df1
        WHERE score1 IS NOT NULL"""

df2 = ps.sqldf(q2)
print(len(df2))

#save total processed 538 data into a single file in the SPI 
df2.to_csv("538/spi_processed.csv")


9662


In [26]:
from datetime import datetime
## Binding 538 and Understat data
teams_understat = understat_data[['HomeID','HomeTeam']]
teams_understat = teams_understat.drop_duplicates()
teams_us = teams_understat[['HomeTeam']]
teams_us = teams_us["HomeTeam"].tolist()

teams_538 = df2[['team1']]
teams_538 = teams_538.drop_duplicates()
teams_538 = teams_538["team1"].tolist()

print(len(teams_us))
print(len(teams_538))
#Check the team names that are the same in both 538 and understat
like_teams = []
for team in teams_538:
    for team1 in teams_us:
        if(team1 == team):
            like_teams.append(team1)
            
print(len(like_teams))
unlike_teams_us = list(set(teams_us) - set(like_teams))
unlike_teams_538 = list(set(teams_538) - set(like_teams))       

#Create a dictionary that maps the team name in understat to the team name in 538 to resolve
#different team names for merging data sets
dict_teams = {}
for u in unlike_teams_us:
    for l in unlike_teams_538:
        try:
            if(u.split(' ')[0] in l):
                dict_teams[l]=u
            elif(l.split(' ')[0] in u):
                dict_teams[l]=u
            elif(l.split(' ')[1] in u):
                dict_teams[l]=u
            elif(u.split(' ')[1] in l):
                dict_teams[l]=u
        except IndexError:
            pass



#Manual additions of the names that did not make the cut through the for loop
dict_teams['SV Darmstadt 98']='Darmstadt'
dict_teams['St Etienne'] = 'Saint-Etienne'
dict_teams['Alavés']='Alaves'
dict_teams['AlavÃ©s']='Alaves'
dict_teams['1. FC Nürnberg']='Nuernberg'
dict_teams['1. FC NÃ¼rnberg']='Nuernberg'
dict_teams['Málaga']='Malaga'
dict_teams['MÃ¡laga']='Malaga'
dict_teams['Deportivo La CoruÃ±a'] = 'Deportivo La Coruna'
dict_teams['Sporting GijÃ³n']='Sporting Gijon'
dict_teams['Spal']='SPAL 2013'
dict_teams['Leeds United'] ='Leeds'
dict_teams['West Ham United']='West Ham'
dict_teams['Brighton and Hove Albion'] = 'Brighton'
dict_teams['Fortuna DÃ¼sseldorf']='Fortuna Duesseldorf'
dict_teams['Hamburg SV'] = 'Hamburger SV'
dict_teams['SC Paderborn'] = 'Paderborn'
dict_teams['SC Freiburg'] = 'Freiburg'
dict_teams['SpVgg Greuther FÃ¼rth']= 'Greuther Fuerth'
dict_teams['F.B.C Unione Venezia'] = 'Venezia'

print(len(dict_teams))

#Check if there are teams that were not catered for by manual insertion and loop 
missed_teams = []
missed_teams1 =[]
missed_teams = list(set(unlike_teams_538)-set(dict_teams.keys()))
missed_teams1 = list(set(unlike_teams_us)-set(dict_teams.values()))
print(missed_teams)
print(missed_teams1)


for ii in like_teams:
    dict_teams[ii]=ii
    
print(dict_teams)



145
145
94
58
[]
[]
{'Hull City': 'Hull', 'TSG Hoffenheim': 'Hoffenheim', 'Hamburg SV': 'Hamburger SV', 'West Ham United': 'West Ham', 'AS Monaco': 'Monaco', 'AS Nancy Lorraine': 'Nancy', 'F.B.C Unione Venezia': 'Venezia', 'AFC Bournemouth': 'Bournemouth', 'Mainz': 'Mainz 05', 'Borussia Monchengladbach': 'Borussia M.Gladbach', 'Bastia': 'SC Bastia', 'SC Freiburg': 'Freiburg', 'SC Paderborn': 'Paderborn', 'RB Leipzig': 'RasenBallsport Leipzig', 'FC Ingolstadt 04': 'Ingolstadt', 'Leeds United': 'Leeds', 'FC Augsburg': 'Augsburg', 'Chievo Verona': 'Chievo', 'Fortuna Düsseldorf': 'Fortuna Duesseldorf', 'St Etienne': 'Saint-Etienne', 'Tottenham Hotspur': 'Tottenham', 'Newcastle': 'Newcastle United', 'Cardiff City': 'Cardiff', 'Huddersfield Town': 'Huddersfield', 'Paris Saint-Germain': 'Paris Saint Germain', 'Norwich City': 'Norwich', 'Deportivo La Coruña': 'Deportivo La Coruna', 'Girona FC': 'Girona', 'Internazionale': 'Inter', 'Parma': 'Parma Calcio 1913', 'Swansea City': 'Swansea', 'Stoke

In [27]:
#create a dictionary for the teams and their respective ids from understat database
id_us = teams_understat[['HomeID']]
id_us = id_us["HomeID"].tolist()
dict_id = dict(zip(teams_us,id_us))


#Check if the key is present in the data
def get_key(dict_a,val):
    for key, value in dict_a.items():
         if val == value:
             return key
 
    return "key doesn't exist"


#Creating a means of combining the matchids in understat and 538 to merge the 2 datasets
understat_data = pd.read_csv("Understat/understat_processed.csv")
dates_understat = understat_data[['Season']]
dates_understat = dates_understat["Season"].tolist()

mid_understat = understat_data[['MatchID']]
mid_understat = mid_understat["MatchID"].tolist()

hid_us = understat_data[['HomeID']]
hid_us = hid_us["HomeID"].tolist()

aid_us = understat_data[['AwayID']]
aid_us = aid_us["AwayID"].tolist()

test_values = []
for ix in range(len(mid_understat)):
    num = str(aid_us[ix])+str(hid_us[ix])+str(dates_understat[ix])
    test_values.append(num)
    
print(len(test_values))
print(len(mid_understat))

#Check that the test values list is a unique list no repeated elements
if len(test_values) > len(set(test_values)):
   print('unique') # do something

dict_mid = dict(zip(test_values,mid_understat))

away_id = []
home_id = []
match_id = []

#Assign team_id to teams and match_id to each entry in the 538 database
with open("538/spi_processed.csv", newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for column in reader:
            team1 = column['team1']
            team2 = column['team2']
            date  = column['season']
            team1_id = dict_id[dict_teams[team1]]
            team2_id = dict_id[dict_teams[team2]]
            test_num = str(team2_id)+str(team1_id)+str(date)
            away_id.append(team2_id)
            home_id.append(team1_id)
            match_id.append(dict_mid[test_num])

final_spi = pd.read_csv("538/spi_processed.csv")

#Insert the team_ids and match_ids into the 538 processed dataset dataframe 
try:
    final_spi.insert(1,"MatchID",match_id)
    final_spi.insert(4, "team1_id", home_id)
    final_spi.insert(6, "team2_id", away_id)
    


except ValueError:
    final_spi.drop('team1_id', inplace=True, axis=1)
    final_spi.drop('team2_id', inplace=True, axis=1)
    final_spi.insert(1,"MatchID",match_id)
    final_spi.insert(4, "team1_id", home_id)
    final_spi.insert(6, "team2_id", away_id)

final_spi.to_csv("untransformed.csv")

9662
9662


In [28]:
#Calculating the distance each team travels for each game
import os

team_ids = list(dict_id.values())

#Create .csv file with team names and team ids 
#CSV file stores the team stadium coordinates, name and capacity
#Data was got manually
if(os.path.exists("Team stadiums.csv") == False):
    df_stadia = pd.DataFrame(teams_us, columns=['Teams'])
    df_stadia.insert(1,"Team_id",team_ids)
    df_stadia.to_csv("Team stadiums.csv")
    
import math

#Create dictionaries linking team_ids and latitude and longitudes 
df_stadia = pd.read_csv("Team stadiums.csv")
latlon = df_stadia[['Latitude','Longitude']]
lat = latlon["Latitude"].tolist()
lon = latlon["Longitude"].tolist()
dict_lat = dict(zip(id_us,lat))
dict_lon = dict(zip(id_us,lon))


#formula to calculate the distance between 2 teams
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
  R = 6371 # Radius of the earth in km
  dLat = math.radians(lat2-lat1)  # deg2rad below
  dLon = math.radians(lon2-lon1)
  
  a=math.sin(dLat/2)*math.sin(dLat/2)+math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dLon/2)*math.sin(dLon/2)
          
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a)) 
  d = R * c # Distance in km
  return d

#Calculate distance between teams for each match
distance = []
with open("untransformed.csv", newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for column in reader:
            team1_id = column['team1_id']
            team2_id = column['team2_id']
            team1_lat = dict_lat[int(team1_id)]
            team1_lon = dict_lon[int(team1_id)]
            team2_lat = dict_lat[int(team2_id)]
            team2_lon = dict_lon[int(team2_id)]
            distance.append(getDistanceFromLatLonInKm(team1_lat,team1_lon,team2_lat,team2_lon))

print(len(distance))
try:
    final_spi.insert(10,"distance(km)",distance)
    
except ValueError:
    final_spi.drop('distance(km)', inplace=True, axis=1)
    final_spi.insert(10, "distance(km)", distance)

final_spi.to_csv("untransformed.csv")

9662


In [29]:
import csv 
import pandas as pd
from datetime import datetime
import bisect
#Calculate distance between teams for each match


temp_spi = pd.read_csv("untransformed.csv")
teams_day = temp_spi[['team1_id','team2_id','date']]
id_1 = teams_day["team1_id"].tolist()
id_2 = teams_day["team2_id"].tolist()
dates = teams_day["date"].tolist()

days1 = []
days2 = []
for y in range(len(id_us)):
    days1.append([o for o, l in enumerate(id_1) if l == id_us[y]])
    days2.append([p for p, m in enumerate(id_2) if m == id_us[y]])


for iii in range(len(days1)):
    for jjj in range(len(days2[iii])):
        bisect.insort(days1[iii],days2[iii][jjj])

dates1 = []

for k in range(len(days1)):
    list1 = []
    for e in range(len(days1[k])):
        list1.append(dates[days1[k][e]])
    dates1.append(list1)


days_btwn_games1 = []

for ki in range(len(dates1)):
    list3 =[]
    for ei in range(len(dates1[ki])):
        if ei > 0:
            date_diff = datetime.strptime(dates1[ki][ei],'%Y-%m-%d')-datetime.strptime(dates1[ki][ei-1],'%Y-%m-%d')
            date_diff = int(date_diff.days)
            list3.append(date_diff)
        else: 
            list3.append(0)        
    days_btwn_games1.append(list3)

for i in range(len(days_btwn_games1)):
    for j in range(len(days_btwn_games1[i])):
        if(days_btwn_games1[i][j] > 35):
            days_btwn_games1[i][j]=0

dict_days = dict(zip(id_us,days_btwn_games1))

days_games_team1 = [0]*len(id_1)
days_games_team2 =[0]*len(id_2)
for i in range(len(id_us)):
    z = 0
    for j in range(len(days1[i])):
        if(id_us[i] == id_1[days1[i][j]]):
            days_games_team1[days1[i][j]]=dict_days[id_us[i]][z]
            z+=1
        elif(id_us[i] == id_2[days1[i][j]]):
            days_games_team2[days1[i][j]]=dict_days[id_us[i]][z]
            z+=1

try:
    final_spi.insert(10,"days_home",days_games_team1)
    final_spi.insert(11,"days_away",days_games_team2)
    
except ValueError:
    final_spi.drop('days_home', inplace=True, axis=1)
    final_spi.drop('days_away', inplace=True, axis=1)
    final_spi.insert(10,"days_home",days_games_team1)
    final_spi.insert(11,"days_away",days_games_team2)

final_spi.to_csv("untransformed.csv")


In [30]:
import pandas as pd

#merge the two datasets from 538 and understat
result = pd.merge(final_spi, understat_data, on="MatchID")
display(result)
result.to_csv("untransformed.csv")

##Remove unnecessary columns
untransformed = pd.read_csv("untransformed.csv")
l = untransformed.columns
display(l)
new_ut = untransformed[['MatchID', 'season', 'team1_id',
       'league_id', 'team2_id', 'league', 'team1', 'team2','days_home', 'days_away',
        'distance(km)','spi1', 'spi2','prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'home_goals', 'away_goals', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2','HomeID','HomeTeam','Home_short','AwayID','AwayTeam','Away_short',
       'home_xG','away_xG', 'datetime', 'w','Results', 'd', 'l']]

cols = ['MatchID', 'season', 'datetime', 'league_id','league', 
        'HomeID', 'HomeTeam','Home_short','days_home',
        'AwayID', 'AwayTeam','Away_short','days_away',
        'distance(km)',
        'spi1', 'spi2','proj_score1', 'proj_score2','importance1', 'importance2', 
        'xg1', 'xg2', 'nsxg1','nsxg2', 'adj_score1', 'adj_score2','home_xG','away_xG',
        'home_goals','away_goals','Results']

new_ut = new_ut[cols]
new_ut.to_csv("untransformed_1.csv")


,Unnamed: 0_x,MatchID,season,date,team1_id,league_id,team2_id,league,team1,team2,...,Away_short,home_goals,away_goals,home_xG,away_xG,datetime,w,d,l,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1
0,0,2249,2016,2016-08-12,162,1843,161,French Ligue 1,Bastia,Paris Saint-Germain,...,PSG,0,1,0.828385,0.664887,2016-08-12 22:00:00,0.3629,0.3789,0.2582,0.0
1,1,2250,2016,2016-08-12,171,1843,169,French Ligue 1,AS Monaco,Guingamp,...,GUI,2,2,2.496410,0.699022,2016-08-12 22:30:00,0.8310,0.1269,0.0421,1.0
2,2,461,2016,2016-08-13,91,2411,75,Barclays Premier League,Hull City,Leicester City,...,LEI,2,1,0.740018,2.456310,2016-08-13 15:30:00,0.0538,0.1247,0.8215,0.0
3,3,462,2016,2016-08-13,78,2411,76,Barclays Premier League,Crystal Palace,West Bromwich Albion,...,WBA,0,1,0.990062,0.684593,2016-08-13 18:00:00,0.4295,0.3356,0.2349,1.0
4,4,463,2016,2016-08-13,72,2411,82,Barclays Premier League,Everton,Tottenham Hotspur,...,TOT,1,1,0.790063,1.123480,2016-08-13 18:00:00,0.2313,0.3322,0.4365,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9657,9657,16879,2021,2021-11-21,101,1854,95,Italy Serie A,Genoa,AS Roma,...,ROM,0,2,0.102684,2.342720,2021-11-21 19:45:00,0.0057,0.0710,0.9233,NaN
9658,9658,17272,2021,2021-11-21,140,1869,146,Spanish Primera Division,Real Sociedad,Valencia,...,VAL,0,0,0.405372,0.584859,2021-11-21 20:00:00,0.2072,0.4519,0.3409,NaN
9659,9659,16880,2021,2021-11-22,94,1854,108,Italy Serie A,Verona,Empoli,...,EMP,2,1,1.302310,1.562630,2021-11-22 17:30:00,0.2672,0.2850,0.4478,NaN
9660,9660,16885,2021,2021-11-22,113,1854,99,Italy Serie A,Torino,Udinese,...,UDI,2,1,1.935660,1.002940,2021-11-22 19:45:00,0.6475,0.2286,0.1239,NaN


Index(['Unnamed: 0', 'Unnamed: 0_x', 'MatchID', 'season', 'date', 'team1_id',
       'league_id', 'team2_id', 'league', 'team1', 'team2', 'days_home',
       'days_away', 'distance(km)', 'spi1', 'spi2', 'prob1', 'prob2',
       'probtie', 'proj_score1', 'proj_score2', 'importance1', 'importance2',
       'score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1',
       'adj_score2', 'Unnamed: 0_y', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
       'Season', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed